In [1]:
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sys.path.append("./utils")

In [2]:
import loader
import round_classifier
import thresholds
from Parser import Parser

In [3]:
all_data = loader.load_dataset()

In [4]:
all_columns = all_data.columns
all_columns[0:16]

Index(['time_left', 'ct_score', 't_score', 'map', 'bomb_planted', 'ct_health',
       't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets',
       't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive'],
      dtype='object')

In [5]:
parser = Parser()
# treat_data = all_data.copy()
treat_data = parser.classify_weapons(all_data.copy())
# treat_data = parser.create_round_winner_columns(treat_data.copy())

In [6]:
treat_data["pistol_round"] = treat_data.apply(
    round_classifier.define_pistol_round, axis=1
)

# Defining eco rounds
treat_data["ct_eco"] = treat_data.apply(
    round_classifier.define_full_ct_eco, axis=1)
treat_data["t_eco"] = treat_data.apply(
    round_classifier.define_full_t_eco, axis=1)

# Defining force buy rounds
treat_data["ct_force"] = treat_data.apply(
    round_classifier.define_force_ct_round, axis=1
)
treat_data["t_force"] = treat_data.apply(
    round_classifier.define_force_t_round, axis=1)


In [7]:
# Selecting columns
normal_columns = all_columns[0:16].tolist()
made_columns_t = [
    "t_main_rifle",
    "t_sec_rifle",
    "t_force_weapons",
    "t_weak_pistols",
    "t_strong_pistols",
    "t_granades",
    "t_weapon_awp",
    "t_eco",
    "t_force",
]
made_columns_ct = [
    "ct_main_rifle",
    "ct_sec_rifle",
    "ct_force_weapons",
    "ct_weak_pistols",
    "ct_strong_pistols",
    "ct_granades",
    "ct_weapon_awp",
    "ct_eco",
    "ct_force",
]
extra_columns = [
    "pistol_round",
    "round_winner",
    # "round_winner_t",
    # "round_winner_ct",
    # "t_health_thresholds",
    # "t_health_thresholds_num",
    # "ct_health_thresholds",
    # "ct_health_thresholds_num",
]
selected_columns = normal_columns + made_columns_ct + made_columns_t + extra_columns

# Getting only columns selected columns
final_df = treat_data[selected_columns]
# checando nans
final_df

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_sec_rifle,t_force_weapons,t_weak_pistols,t_strong_pistols,t_granades,t_weapon_awp,t_eco,t_force,pistol_round,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0,0,1,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0,0,1,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,True,200.0,242.0,195.0,359.0,100.0,...,0.0,0.0,2.0,0.0,3.0,1.0,0,0,0,T
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0,0,0,T
122407,114.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,0.0,0.0,0.0,1.0,14.0,1.0,0,0,0,T
122408,94.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,0.0,0.0,0.0,1.0,12.0,1.0,0,0,0,T


In [8]:
final_df.dtypes

time_left            float64
ct_score             float64
t_score              float64
map                   object
bomb_planted            bool
ct_health            float64
t_health             float64
ct_armor             float64
t_armor              float64
ct_money             float64
t_money              float64
ct_helmets           float64
t_helmets            float64
ct_defuse_kits       float64
ct_players_alive     float64
t_players_alive      float64
ct_main_rifle        float64
ct_sec_rifle         float64
ct_force_weapons     float64
ct_weak_pistols      float64
ct_strong_pistols    float64
ct_granades          float64
ct_weapon_awp        float64
ct_eco                 int64
ct_force               int64
t_main_rifle         float64
t_sec_rifle          float64
t_force_weapons      float64
t_weak_pistols       float64
t_strong_pistols     float64
t_granades           float64
t_weapon_awp         float64
t_eco                  int64
t_force                int64
pistol_round  

In [9]:
def round_winner_binnary(dataframe):
    round_winner_bin=[]
    for i in range(0,len(dataframe.index)):
        if dataframe['round_winner'].iloc[i] == 'CT':
            round_winner_bin.append(1)
        elif dataframe['round_winner'].iloc[i] == 'T':
            round_winner_bin.append(0)
    dataframe['roundwinner_bin']=round_winner_bin

round_winner_binnary(final_df)

final_df.head()

C:\Users\luca\AppData\Local\Temp/ipykernel_25068/1201939889.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['roundwinner_bin']=round_winner_bin


,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_force_weapons,t_weak_pistols,t_strong_pistols,t_granades,t_weapon_awp,t_eco,t_force,pistol_round,round_winner,roundwinner_bin
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,CT,1
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,0.0,2.0,0.0,0,0,1,CT,1
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,2.0,0.0,0,0,1,CT,1
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,CT,1
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,1,0,0,CT,1


In [10]:
from sklearn.metrics import mean_squared_error,confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

labels=final_df['roundwinner_bin'].copy()


In [11]:
cat_attribs = ["map", "bomb_planted", "ct_eco", "ct_force", "t_eco", "t_force", "pistol_round", 'ct_helmets', 't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive', 'ct_main_rifle',
               'ct_sec_rifle', 'ct_force_weapons', 'ct_weak_pistols', 'ct_strong_pistols', 'ct_weapon_awp', 't_main_rifle', 't_sec_rifle', 't_force_weapons', 't_weak_pistols', 't_strong_pistols', 't_weapon_awp']

for col in cat_attribs:
    final_df[col] = final_df[col].astype('category')

C:\Users\luca\AppData\Local\Temp/ipykernel_25068/1664102152.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[col] = final_df[col].astype('category')


In [12]:
# meu_imputer = SimpleImputer(strategy="median")
meu_scaler = StandardScaler()

num_pipeline = Pipeline(
    [ ("std_scaler", meu_scaler), ])

meu_one_hot_encoder = OneHotEncoder(sparse=False)

cat_pipeline = Pipeline([("cat_encoder", meu_one_hot_encoder), ])


dataset_num = final_df.drop(labels=["map", "bomb_planted", "ct_eco", "ct_force", "t_eco", "t_force", "pistol_round", 'ct_helmets', 't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive', 'ct_main_rifle',
                                    'ct_sec_rifle', 'ct_force_weapons', 'ct_weak_pistols', 'ct_strong_pistols', 'ct_weapon_awp', 't_main_rifle', 't_sec_rifle', 't_force_weapons', 't_weak_pistols', 't_strong_pistols', 't_weapon_awp', "round_winner", 'roundwinner_bin'], axis=1)
num_attribs = list(dataset_num)
cat_attribs = ["map", "bomb_planted", "ct_eco", "ct_force", "t_eco", "t_force", "pistol_round", 'ct_helmets', 't_helmets', 'ct_players_alive', 't_players_alive', 'ct_force_weapons', 'ct_weak_pistols', 'ct_strong_pistols', 'ct_weapon_awp', 't_main_rifle', 't_sec_rifle', 't_force_weapons', 't_weak_pistols', 't_strong_pistols', 't_weapon_awp']

full_pipeline = ColumnTransformer(
    [("num", num_pipeline, num_attribs), ("cat", cat_pipeline, cat_attribs), ]
)
x_raw = final_df.drop(columns=['round_winner', 'roundwinner_bin'])

dataset_prepared = full_pipeline.fit_transform(x_raw)


In [13]:
dataset_num

,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_granades,t_granades
0,175.00,0.0,0.0,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0
1,156.03,0.0,0.0,500.0,500.0,400.0,300.0,600.0,650.0,0.0,2.0
2,96.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,2.0
3,76.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0
4,174.97,1.0,0.0,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,3.0
122406,174.93,11.0,15.0,500.0,500.0,95.0,175.0,11500.0,23900.0,2.0,2.0
122407,114.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,10.0,14.0
122408,94.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,12.0


In [14]:
from sklearn.model_selection import train_test_split
seed = 42
X = dataset_prepared
X_train, X_test, y_train, y_test = train_test_split(X,labels,test_size=0.333, random_state = seed)
X_t, X_te, y_t, y_te = train_test_split(x_raw,labels,test_size=0.333, random_state = seed)

## Random forest

In [41]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(
                      n_estimators=350,
                      n_jobs=-1,
                      random_state=seed,
                      max_features='auto')
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=450, n_jobs=-1, random_state=42)

In [51]:
from sklearn.model_selection import cross_val_score
cross_val_score(
    rf_classifier,
    X_train,
    y_train,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
)

array([0.84086567, 0.84997795, 0.84449752])

In [52]:
y_pred = rf_classifier.predict(X_test)
y_pred

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [53]:
confusion_matrix(y_test,y_pred)

array([[17979,  2715],
       [ 2554, 17515]], dtype=int64)

In [54]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.8707406226234575
Precision: 0.865793376173999
Recall: 0.872739050276546


In [49]:
import numpy as np
import plotly.express as px
cat_cols = full_pipeline.transformers_[1][1]["cat_encoder"].get_feature_names(
    cat_attribs
)
num_cols = num_attribs
cols = np.concatenate((num_cols, cat_cols))
print(len(cols), len(rf_classifier.feature_importances_))
df_feat_importances = pd.DataFrame(
    {"Stats": cols, "FI": rf_classifier.feature_importances_}
).sort_values(by="FI", ascending=0)
df_feat_importances.head(20)
fig = px.bar_polar(
    df_feat_importances.iloc[:20],
    r="FI",
    theta="Stats",
    color="Stats",
    template="plotly_dark",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
)
fig.show()


116 116


## SGD Classifier

In [22]:
from sklearn.linear_model import SGDClassifier

# Existe aleatoriedade dentro do SGDClassifier, por isso o argumento
RANDOM_SEED = 420
sgd_clf = SGDClassifier(
    max_iter=500,
    tol=1e-3,
    random_state=RANDOM_SEED,
)
sgd_clf.fit(X_train, y_train)
y_pred_sgd = rf_classifier.predict(X_test)
y_pred_sgd

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [23]:
from sklearn.model_selection import cross_val_score
cross_val_score(
    sgd_clf,
    X_train,
    y_train,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
)

array([0.74775867, 0.74875073, 0.74870476])

In [24]:
confusion_matrix(y_test,y_pred_sgd)

array([[15704,  4990],
       [ 4171, 15898]], dtype=int64)

In [25]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_sgd))
print("Precision:",metrics.precision_score(y_test, y_pred_sgd))
print("Recall:",metrics.recall_score(y_test, y_pred_sgd))

Accuracy: 0.7752618796457572
Precision: 0.7611068556108771
Recall: 0.7921670237680004


## Naive Bayes

In [26]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gaussianN = GaussianNB()

# Train the gaussianN using the training sets
gaussianN.fit(X_train, y_train)

#Predict Output
predicted= gaussianN.predict(X_test) # 0:Overcast, 2:Mild
predicted

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [27]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision:",metrics.precision_score(y_test, predicted))
print("Recall:",metrics.recall_score(y_test, predicted))

Accuracy: 0.7117974633859137
Precision: 0.6713761996951847
Recall: 0.8121480890926305
